In [1]:
!pip install -q accelerate datasets peft bitsandbytes tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 24.6 MB/s eta 0:00:00:00:0100:01


In [20]:
!wget -q -O smolvlm.py https://raw.githubusercontent.com/blancsw/deep_4_all/refs/heads/main/cours/TP/smolvlm.py
!cat smolvlm.py

import torch
from datasets import load_dataset
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoProcessor, BitsAndBytesConfig, Idefics3ForConditionalGeneration, Trainer, TrainingArguments

USE_QLORA = False

model_id = "HuggingFaceTB/SmolVLM-256M-Instruct"


def main():
    ds = load_dataset('merve/vqav2-small', trust_remote_code=True)
    split_ds = ds["validation"].train_test_split(test_size=0.5)
    train_ds = split_ds["train"]
    processor = AutoProcessor.from_pretrained(
            model_id
            )
    lora_config = LoraConfig(
            r=8,
            lora_alpha=8,
            lora_dropout=0.1,
            # nous entrainons seulment les linear layer du multi head attention
            # https://github.com/huggingface/transformers/blob/d3af76df58476830eb5b5981decc64af15e369f5/src/transformers/models/llama/modeling_llama.py#L254
            target_modules=['down_proj', 'o_proj', 'k_proj', 'q_proj', 'gate_proj', 'u

In [21]:
!accelerate launch --multi_gpu --mixed_precision=fp16 --num_processes=2 smolvlm.py

2025-02-05 20:43:08.985202: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-05 20:43:08.987225: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-05 20:43:09.008822: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-05 20:43:09.009222: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-05 20:43:09.015578: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory